In [23]:
import insightface
import urllib
import urllib.request
import cv2
import numpy as np

def url_to_image(url):
    resp = urllib.request.urlopen(url)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    return image

In [83]:
url = 'https://dujye7n3e5wjl.cloudfront.net/photographs/1080-tall/time-100-influential-photos-ellen-degeneres-oscars-selfie-100.jpg'
img = url_to_image(url)

In [3]:
insightface.model_zoo.get_model_list()

['arcface_mfn_v1',
 'arcface_r100_v1',
 'genderage_v1',
 'retinaface_mnet025_v1',
 'retinaface_mnet025_v2',
 'retinaface_r50_v1']

In [75]:
model = insightface.app.FaceAnalysis(rec_name = 'arcface_r100_v1', det_name = 'retinaface_mnet025_v2')

In [77]:
ctx_id = -1
model.prepare(ctx_id = ctx_id, nms=0.4)

[32, 16, 8] {'32': {'SCALES': (32, 16), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '16': {'SCALES': (8, 4), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '8': {'SCALES': (2, 1), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}}
use_landmarks True


In [91]:
import time
start = time.time()
emb_list = []

img_out = img.copy()

faces = model.get(img)
for idx, face in enumerate(faces):
    print("Face [%d]:"%idx)
    print("\tage:%d"%(face.age))
    gender = 'Male'
    if face.gender==0:
        gender = 'Female'
    print("\tgender:%s"%(gender))
    print("\tembedding shape:%s"%face.embedding.shape)
    emb_list.append(np.expand_dims(face.embedding, axis=0))
    bboxes = face.bbox.astype(np.int)
    print("\tbbox:%s"%(bboxes.flatten()))
    img_out = cv2.rectangle(img_out, (bboxes[0], bboxes[1]), (bboxes[2], bboxes[3]), (0, 255, 0), 2)
    print("\tlandmark:%s"%(face.landmark.astype(np.int).flatten()))
    print("")
    img_out = cv2.putText(img_out, str(face.age), (int(bboxes[0]-10), int(bboxes[1]-10)), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 255, 0),2)
    img_out = cv2.putText(img_out, str(gender), (int(bboxes[0]+40), int(bboxes[1]-10)), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 0, 255),2)
    img_out = cv2.putText(img_out, str(idx), (int(bboxes[0]-40), int(bboxes[1]-10)), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (0, 0, 255),2)
    img_out = cv2.putText(img_out, str(round(distncs[0][idx], 2)), (int(bboxes[0]+40), int(bboxes[1]+100)), cv2.FONT_HERSHEY_SIMPLEX, 1.1, (255, 0, 0),2)
cv2.imwrite('out.jpg',img_out)
end = time.time()
print(end - start)

Face [0]:
	age:34
	gender:Female
	embedding shape:512
	bbox:[1422  290 1667  626]
	landmark:[1471  431 1584  416 1524  482 1485  534 1600  521]

Face [1]:
	age:47
	gender:Male
	embedding shape:512
	bbox:[1128  -36 1327  244]
	landmark:[1167   75 1258   73 1203  125 1172  168 1258  166]

Face [2]:
	age:25
	gender:Female
	embedding shape:512
	bbox:[191 143 487 476]
	landmark:[314 250 422 309 347 329 258 352 360 403]

Face [3]:
	age:54
	gender:Female
	embedding shape:512
	bbox:[423 356 646 661]
	landmark:[480 480 583 457 544 515 506 585 601 564]

Face [4]:
	age:32
	gender:Female
	embedding shape:512
	bbox:[669  75 853 343]
	landmark:[705 180 788 186 732 238 698 263 786 268]

Face [5]:
	age:40
	gender:Female
	embedding shape:512
	bbox:[626 416 880 784]
	landmark:[694 567 814 561 758 649 691 675 821 670]

Face [6]:
	age:42
	gender:Male
	embedding shape:512
	bbox:[1501   87 1705  321]
	landmark:[1534  182 1627  174 1568  206 1551  263 1624  256]

Face [7]:
	age:59
	gender:Male
	embedding sha

In [78]:
start = time.time()
img = cv2.imread('bradley.jpg')
faces = model.get(img)
for idx, face in enumerate(faces):
    print("Face [%d]:"%idx)
    print("\tage:%d"%(face.age))
    bradley_emb = np.expand_dims(face.embedding, axis=0)
end = time.time()
print(end - start)

Face [0]:
	age:50
0.9739913940429688


In [101]:
distncs = distance.cdist(bradley_emb, np.array(emb_list*10000).squeeze(), 'cosine')

In [72]:
np.expand_dims(np.array(emb_list).squeeze(),0)

(1, 128)

In [96]:
np.array(emb_list*10000).squeeze().shape

(120000, 512)